#Speech to image Generation App deployment

In [1]:
!pip install -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 10.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803320 sha256=3066214f71882eb6bb2cdf383d7945b27ce26705f9e80446866e5aa6466c1903
  Stored in directory: /root/.cache/pip/wheels/dd/4a/1f/d1c4bf3b9133c8168fe617ed979cab7b14fe381d059ffb9d83
Successfully built openai-whisper


In [2]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-izv8wtnw
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-izv8wtnw
  Resolved https://github.com/openai/whisper.git to commit 173ff7dd1d9fb1c4fddea0d41d704cfefeb8908c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
!pip install ffmpeg-python

In [4]:
!pip install diffusers transformers scipy ftfy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.2 MB/s eta 0:00:00


In [5]:
!pip install transformers torch torchvision pillow

In [11]:
!pip install streamlit-webrtc

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/operations/check.py", line 101, in check_install_conflicts
    package_set, _ = create_package_set_from_installed()
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/operations/check.py", line 39, in create_package_set_from_installed
    for dist in 

In [7]:
!pip install streamlit
!pip install pyngrok

In [ ]:
%%writefile app.py
import streamlit as st
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
import whisper
import torch
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
from PIL import Image
import tempfile
import ffmpeg

# Initialize models
@st.cache_resource
def load_whisper_model():
    return whisper.load_model("medium")

@st.cache_resource
def load_openjourney_models():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    pipe = StableDiffusionPipeline.from_pretrained("prompthero/openjourney-v4", torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32).to(device)
    img2img_pipe = StableDiffusionImg2ImgPipeline.from_pretrained("prompthero/openjourney-v4", torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32).to(device)
    return pipe, img2img_pipe

whisper_model = load_whisper_model()
pipe, img2img_pipe = load_openjourney_models()

# JavaScript-based audio recording
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})

var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

# Record audio
def record_audio(sec=5):
    display(Javascript(RECORD))
    s = output.eval_js(f'record({sec * 1000})')
    b = b64decode(s.split(',')[1])
    return b

# Save audio as WAV
def save_as_wav(audio_data, filename="audio.wav"):
    process = (
        ffmpeg
        .input('pipe:', format='webm')
        .output(filename, format='wav', acodec='pcm_s16le', ac=1, ar='16k')
        .overwrite_output()
        .run_async(pipe_stdin=True)
    )
    process.stdin.write(audio_data)
    process.stdin.close()
    process.wait()

# Functions for processing audio and generating images
def speech_to_trans_text(audio_path):
    result = whisper_model.transcribe(audio_path, task="translate")
    st.write(f"Transcription: {result['text']}")
    return result['text']

def generate_sub_sentences(text, beam_size):
    words = text.split()
    return [' '.join(words[i:i + beam_size]) for i in range(0, len(words), beam_size)]

def generate_image(prompt, image=None, num_inference_steps=30, height=512, width=512):
    if image is None:
        image = pipe(prompt, num_inference_steps=num_inference_steps, height=height, width=width).images[0]
    else:
        image = img2img_pipe(prompt, image=image, num_inference_steps=num_inference_steps, height=height, width=width).images[0]
    return image

# Streamlit App
st.title("Audio-to-Image Pipeline")
st.write("Record audio, transcribe it, and generate images based on the transcription.")

# Step 1: Record Audio
st.header("Step 1: Record Audio")
if st.button("Start Recording"):
    st.write("Recording... Please wait for 5 seconds.")
    # This button triggers the recording using JS code
    audio_data = record_audio(5)  # Record for 5 seconds
    save_as_wav(audio_data, filename="recorded_audio.wav")
    st.success("Recording saved successfully! Please upload the recorded file for processing.")

    # Upload file after recording
    audio_file = st.file_uploader("Upload the recorded audio file", type=["wav"])

    if audio_file is not None:
        with tempfile.NamedTemporaryFile(delete=False) as tmpfile:
            tmpfile.write(audio_file.getvalue())
            audio_path = tmpfile.name
            st.audio(audio_path, format="audio/wav")
            st.success("Audio file uploaded successfully!")

# Step 2: Process Audio and Generate Images
if audio_path and st.button("Process Transcription and Generate Images"):
    with st.spinner("Processing audio..."):
        transcription = speech_to_trans_text(audio_path)

    if transcription:
        beam_size = st.slider("Set Beam Size for Sub-sentences", min_value=1, max_value=10, value=3)
        sub_sentences = generate_sub_sentences(transcription, beam_size)

        first_image = None
        for idx, prompt in enumerate(sub_sentences):
            st.write(f"Prompt: {prompt}")
            with st.spinner(f"Generating image for: {prompt}"):
                image = generate_image(prompt, first_image)
                st.image(image, caption=f"Generated Image {idx + 1}")
                first_image = image

        st.success("All images generated successfully!")


In [9]:
from pyngrok import ngrok

!ngrok config add-authtoken "2pDRV3BJSTo22NZ7SYbriKfOb19_4W1eQG6b9oz7jQAEDX33g"

public_url = ngrok.connect(addr=8503)
print("Streamlit app URL:", public_url)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Streamlit app URL: NgrokTunnel: "https://df26-34-80-17-197.ngrok-free.app" -> "http://localhost:8503"


In [10]:
!streamlit run app.py &>/content/logs.txt &